# AME (4,6) with qudit gates

In [1]:
import numpy as np
import qutip
from qhexagates import QHexaGates 


In [10]:

# Helper functions for complex exponentials
def exp(angle):
    return np.exp(2j * np.pi * angle / 6)

def exp_3(angle):
    return np.exp(2j * np.pi * angle / 3)

# Initialize the class and basic operators
qhex = QHexaGates()
I = np.identity(6)
q0 = qhex.q[0]

# Prepare initial state |0000⟩
initial_state = np.kron(np.kron(np.kron(q0, q0), q0), q0)

# Apply Fourier gates on quhexas 0 and 1
F0 = np.kron(qhex.fourier(), np.kron(I, np.kron(I, I)))
F1 = np.kron(I, np.kron(qhex.fourier(), np.kron(I, I)))

psi = initial_state @ F0 @ F1
psi = psi @ qhex.CNOT_initial_state(k=0, j=2)
psi = psi @ qhex.CNOT_initial_state(k=1, j=3)

# Apply generalized CNOT and another Fourier on quhexa 0
psi = psi @ qhex.P_gate_01()
psi = psi @ F0

# Define Λ₀ (Lambda0): controlled phase gates on target quhexa 2
Lambda0 = (
    qhex.controlled_01(0, [exp(0), exp(1), exp(0), exp(1), exp(3), exp(3)]) @
    qhex.controlled_01(1, [exp(3), exp(3), exp(1), exp(5), exp(2), exp(4)]) @
    qhex.controlled_01(2, [exp(2), exp(1), exp(3), exp(1), exp(2), exp(3)]) @
    qhex.controlled_01(3, [exp(1), exp(1), exp(2), exp(0), exp(3), exp(5)]) @
    qhex.controlled_01(4, [exp(5), exp(3), exp(2), exp(3), exp(2), exp(5)]) @
    qhex.controlled_01(5, [exp(4), exp(4), exp(1), exp(5), exp(5), exp(1)])
)

# Define Λ₁ (Lambda1)
Lambda1 = (
    qhex.controlled_01(0, [exp(0), exp(2), exp(3), exp(3), exp(2), exp(0)]) @
    qhex.controlled_01(1, [exp(0), exp(3), exp(2), exp(2), exp(0), exp(4)]) @
    qhex.controlled_01(2, [exp(2), exp(0), exp(3), exp(5), exp(0), exp(0)]) @
    qhex.controlled_01(3, [exp(0), exp(5), exp(0), exp(0), exp(2), exp(0)]) @
    qhex.controlled_01(4, [exp(2), exp(2), exp(5), exp(3), exp(2), exp(4)]) @
    qhex.controlled_01(5, [exp(2), exp(3), exp(0), exp(2), exp(0), exp(0)])
)

# Define Λ₂ (Lambda2) using exp_3
Lambda2 = (
    qhex.controlled_01(0, [exp_3(0), exp_3(2), exp_3(2), exp_3(0), exp_3(0), exp_3(1)]) @
    qhex.controlled_01(1, [exp_3(0), exp_3(1), exp_3(1), exp_3(1), exp_3(2), exp_3(1)]) @
    qhex.controlled_01(2, [exp_3(0), exp_3(2), exp_3(0), exp_3(2), exp_3(2), exp_3(2)]) @
    qhex.controlled_01(3, [exp_3(2), exp_3(0), exp_3(2), exp_3(2), exp_3(2), exp_3(1)]) @
    qhex.controlled_01(4, [exp_3(1), exp_3(1), exp_3(2), exp_3(0), exp_3(2), exp_3(2)]) @
    qhex.controlled_01(5, [exp_3(0), exp_3(1), exp_3(2), exp_3(2), exp_3(1), exp_3(0)])
)

# Apply Λ₀ (can replace with Λ₁ or Λ₂ to generate different AME states)
psi = psi @ Lambda2

# Final operations
psi = psi @ F0
psi = psi @ qhex.P_gate_01()


In [11]:
ame_state_normalized = psi/np.linalg.norm(psi)

## Compute the entropy to check if it is an AME state

In [12]:
wfn_array = np.array(ame_state_normalized[0])
d = 6
nqudits = 4
wfn = qutip.Qobj(wfn_array, dims=[[d] * nqudits, [1]*nqudits])
print(wfn)

Quantum object: dims = [[6, 6, 6, 6], [1, 1, 1, 1]], shape = (1296, 1), type = ket
Qobj data =
[[0.08333333+1.23358114e-17j]
 [0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j]
 ...
 [0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j]]


In [15]:
N_A = {'AB|CD': [0,1], 'AC|BD':[0,2],'AD|BC': [0,3]} #all possible bipartitions
for key, n in N_A.items():
    partial_rho = wfn.ptrace(n)
    eigval_rho = np.linalg.eigvals(partial_rho.full())

    for k in range(len(eigval_rho)):
        if eigval_rho[k] < 0.00001:
            eigval_rho[k] = 1

    print(f'Entropy of the partition {key}: {-np.real(eigval_rho.dot(np.emath.logn(d, eigval_rho)))}')

Entropy of the partition AB|CD: 2.0
Entropy of the partition AC|BD: 2.0
Entropy of the partition AD|BC: 1.9999999999999993
